# 5 情报收集

“情报”指的是目标网络、服务器、应用程序的所有信息。渗透测试人员需要使资源尽可能地获取要测试目标的相关信息 如果现在采用了黑盒测试的方式，那这个阶段可以说是整个渗透测试过程中最为重要的一个阶段。所谓“知己知彼，百战不殆”也正是说明了情报收集的重要性。

## 5.1 信息收集基础

信息收集获得信息的方法可以分成两种:被动扫描和主动扫描。
### 被动扫描

被动扫描主要指的是在**目标无法察觉**的情况下进行的信息收集。

### 主动扫描
主动扫描一般都是针对目标发送特制数据包，然后根据目标的反应来获得一些信息。

扫描之后将会获得的信息包括：目标网络的结构，目标网络，所使用设备的类型，目标主机上运行的操作系统，目标主机上所开放的端口，目标主机上所提供的服务，目标主机上所运行的应用程序

选择扫描目标的 nmap 语法如下所示：
```bash
1)扫描指定 IP 主机： nmap 192.168.169.133
2)扫描指定域名主机:  nmap www.nmap.com
3)扫描指定范围主机:  nmap 192.168.169.1-20
4)扫描一个子网主机:  nmap 192.168.169.0/24
```
对目标的端口进行扫描的 nmap 语法如下所示：
```bash
1)扫描一个主机的特定端口 ：nmap -p 22 192.168.169.1
2)扫描指定范围端口： nmap -p 1-80 192.168.169.1
3)扫描 100 个最为常用的端口:  nmap -F 192.168.169.1
```
对目标端口状态进行扫描的 nmap 语法如下所示:
```bash
1)使用 TCP 全开扫描： nmap -sT 192.168.169.1
2)使用 TCP 半开扫描： nmap -sS 192.168.169.1
3)使用 UDP 扫描： nmap -sU -p 123,161,162 192.168.169.1
```
对目标的操作系统和运行服务进行扫描的 nmap 语法如下所示:
```bash
1)扫描目标主机上运行的操作系统： nmap -O 192.168.169.1
2)扫描目标主机上运行的服务类型： nmap -sV 192.168.169.1
```

## 5.2 主机状态扫描

处于运行状态而且网络功能正常的主机被称为活跃主机，反之则称为非活跃主机。

那么进行扫描的原理是什么呢？

这里有一个 TEST 协议（这个协议目前并不存在，仅用于举例，这里假设主机A和主机B都遵守这个协议），它规定了如果一台主机B收到了来自于A的格式为“请求”的数据包，那么它必须在一定时间内向主机A再发送一个格式为“回应”的数据包（实际上这个过程在很多真实的网络协议中都存在）

那么，如果现在想知道主机B是否为活跃主机，你该知道怎么办了吧？只需要在你的主机A上构造一个“请求”，然后将它发送给主机B如果主机B是活跃主机，那么你就会收到来自它的“回应”数据包，否则什么都收不到。

### 5.2.1 基于 ARP 的活跃主机发现技术

ARP 的中文名字是 “地址解析协议”，主要用在以太网中。

当所使用的主机只知道目标的主机地址却不知道目标的硬件地址的时候，就需要使用以太广播包给网络上的每一台主机发送 ARP 请求。请求格式如下：
```bash
协议类型 ARP Request (ARP 请求）
源主机 IP 地址： 192.168.1.1
目标主机 IP 地址: 192.168.1.2
源主机硬件地址: 11:11:11:11:11:11 
目标主机 MAC 地址: ff:ff:ff:ff:ff:ff
```
当交换机网络中的其余主机在接收到这个ARP请求数据包之后，它会用自己的 IP 地址与包中头部的目标主机 IP 地址相比较，如果不匹配，就不会做出回应。 例如，192.168.1.3接收到这个数据包时，就使用本身地址和 192.168.1.2 进行比较，如发现不同，不做出回应。如果匹配，例如， 192.168 1.2 收到这个请求，这个设备就会给发送 IP 请求的设备发送一个ARP回应数据包，这个回应的格式如下：
```bash
协议类型 ARP Reply (ARP 回应）
源主机 IP 地址： 192.168.1.2
目标主机 IP 地址： 192.168.1.1 
源主机硬件地址： 22:22:22:22:22:22
目标主机 MAC 地址： 11:11:11:11:11:11
```



当目标主机与我们处于同一以太网的时候，利用 ARP 对其进行扫描是一个最好的选择，因为这种扫描方式最快，也最为精准，没有任何的安全措施会阻止这种扫描方式。过程如下：

第一步：向目标发送一个 ARP Request

第二步： 如果目标主机处于活跃状态，它**一定会回应**一个ARP Reply

第三步：但是如果目标主机处于非活跃状态，它不会给出任何回应。



基于 ARP 扫描是一种最为高效的方法，但是它的局限性也很明显，只能够扫描同一以太网内的主机。 例如 ，主 IP 地址 192.1.169.13 ，子网掩码为255.255.255.0，那么使用 ARP 扫描的范围只能是 192.168.169.1-255。 如果目标的地址为 192.168.1.100 ，这种方法就不适用了。

### 5.2.2 基于 ICMP 的活跃主机发现技术

ICMP 也位于TCP/IP协议族中的网络层，它的目的是用于在IP主机、路由器之间传递控制消息。

相比 ARP 简单明了的工作模式，ICMP 虽然要复杂一些，但是用来扫描活跃主机的原理却是一样的。

ICMP 中提供了多种报文，这些报文又可以分成两大类：差错报文和查询报文。其中，查询报文都是由一个请求和一个应答构成，这一点和之前讲过的 TEST 协议一样， 只需要
向目标发送一个请求数据，如果收到了来自目标的回应，就可以判断目标是活跃主机，否则可以判断目标是非活跃主机，这与 ARP 扫描原理是相同的。

但是，与 ARP 扫描不同的地方在于 ICMP 查询报文有四种，分别是响应请求或应答、时间戳请求或应答、地址掩码请求或应答、路由器询问或应答。 但是在实际应用中，后面三种的成功率很低，所以本节主要采用第一种 ICMP 查询报文。

Ping 命令就是 响应请求或应答 的一种应用，我们经常会使用这个命令来测试本地与目标之间的连通性。例如我们所在的 IP 是 192.168.1.1，而通信的目标 IP 地址为 192.168.1.2。
如果要判断 192.168.1.2 是否为活跃主机， 就需要向其发送一个 ICMP 请求， 这个请求的格式如下：
```bash
IP 层内容
源 IP 地址: 192.168.1.1 
目的 IP 地址: 192.168.1.2 
ICMP 层内容
Type : 8 （表示请求）
```

如果 192.168 1.2 台主机处于活跃状态，它在收到了这个请求之后，就会给出一个回应，这个回应的格式如下所示：
```bash
IP 层内容
源 IP 地址 192.168.1.2
目的 IP 地址 192 168 1.1
ICMP 层内容
Type : 0 （表示应答）
```


也就是说，只要收到了 ICMP 回应 ，就可以判断该主机为活跃状态，即：

第一步：向目标发送一个 ICMP Request

第二步 ：如果目标主机处于活跃状态，在正常情况下它就会回应一个 ICMP Reply

需要注意的是，由于现在很多网络安全设备或者机制会屏蔽 ICMP ，在这种情况下即使
目标主机处于活跃状态，也收不到任何的回应。

第三步：如果目标主机处于非活跃状态，它不会给出任何回应

基于 ICMP 的扫描是一种很常见的方法，相比 ARP 只能应用于以太网环境中的特点，这种方法的应用范围要广泛得多。 无论是以太网还是互联网都可以使用这种方法， 但是基于 ICMP 的扫描的缺陷也很明显，由于大量网络设备，例如很多路由器、防火墙等都 ICMP 进行了屏蔽，这样就会导致扫描结果不准确。

### 5.2.3 基于 TCP 的活跃主机发现技术

TCP和ARP、ICMP 等协议并不处于同一层，而是位于它们的上一层传输层。在这一层中出现了“端口”的概念。 “端口” 是英文 port 意译，可以认为是设备与外界通信交流
出口 端口可分为虚拟端口和物理端口，这里使用的就是虚拟端口，指的是计算机内部或交换机路由器内的端口，例如计算机中的 80 端口、 21 端口 、23 端口 这些端口可以被不同的服务所使用来进行各种通信，例如 Web 服务、 FTP 服务、 SMTP 务等， 这些服务都是通过“ IP 地址＋端口号”来区分的。

如果检测到一台主机的某个端口有回应，也一样可以判断这台主机是活跃主机。需要注意的是，如果一台主机处于活跃状态，那么它的端口即使是关闭的，在收到请求时，也会
给出一个回应，只不过并不是一个“ SYN+ACK ”数据包，而是一个拒绝连接的“ RST ”数据包。

这样在检测目标主机是否是活跃主机的时候，就可以向目标的 80 端口发送一个 SYN 数据包，之后的情形可能有如下三种：

第一种：主机发送的 SYN 数据包到达了目标的 80 端口处，但是目标端口关闭了，所以目标主机会发回一个 RST 数据包

第二种：主机发送的 SYN 数据包到达目标的 80 端口处，而且目标端口开放，所以目标主机会发回一个“ SYN+ACK ”数据包

第三种：主机发送的 SYN 数据包到达不了目标，这时就不会收到任何回应

也就是说，只要收到了 TCP 回应，就可以判断该主机为活跃状态

### 5.2.4 基于 UDP 的活跃主机发现技术

UDP 全称是用户数据报协议，在网络中它与 TCP 一样用于处理数据包，是一种无连接的协议。但基于 UDP 的活跃主机发现技术和 TCP 不同， UDP 没有3次握手， 当向目标发送一个UDP 数据包之后，目标是不会发回任何 UDP 数据包的。 不过，如果目标主机是处于活跃状态的，但是目标端口是关闭的时候，可以返回一个 ICMP 数据包，这个数据包的含义为“ unreachable ”。如果目标主机不处于活跃状态，这时是收不到任何回应的。

## 5.3 端口扫描

正常的情况下，端口只有 open （开放）和 closed （关闭）两种状态 但是有时网络安全机制会屏蔽对端口的探测，因此端口状态可能会出现无法判断的情况，所以在探测的时候需要为端口加上一个 filtered 状态，表示无法获悉目标端口的真正状态。

在本章只介绍其中最为常用的两种方法： TCP 全开扫描 和 TCP 半开扫描

### 5.3.1 基于 TCP 全开的端口扫描技术

这种扫描的思想很简单，如果目标端口是开放的，那么在接到主机端口发出的 SYN 请求之后，就会返回一个 SYN+ACK 回应，表示愿意接受这次连接的请求，然后主机端口再回应一个 ACK ，这样就成功地和目标端口建立了一个 TCP 连接。如果目标端口是关闭的，那么在接到主机端口发出的 SYN 请求之后，就会返回一个 RST 回应，表示不接受这次连接的请求，这样就中断了这次 TCP 连接。

但是目标端口不开放还有另外一种情况，就是当主机端口发出 SYN 请求之后，没有收到任何的回应。多种原因都可能造成这种情况，例如，目标主机处于非活跃状态，这时当然
无法进行回应，不过这也可以认为端口是关闭的。另外一些网络安全设备也会屏蔽掉对某些端口的 SYN 请求，这时也会出现无法进行回应的情况，在此暂时先不考虑后一种情况。

### 5.3.2 基于 TCP 半开的端口扫描技术

基于 TCP 全开的端口扫描技术还有一些不完善的地方，例如，这次连接可能会被目标主机的日志记录下来，而且最为主要的是建立 TCP 连接三次握手中的最后一次是没用的，在目标返回一个 SYN+ACK 类型的数据包之后，已经达到了探测的目的，最后发送的 ACK 类型数据包是不必要的，所以可以考虑去除这一步。

于是一种新的扫描技术产生了，这种扫描的思想很简单，如果目标端口是开放的，那么在接到主机端口发出的 SYN 请求之后，就会返回一个 SYN+ACK 回应，表示愿意接受这次
连接的请求，然后主机端口不再回应 ACK ，而是发送一个 RST 表示中断这个连接。这样实际上并没有建立好完整的 TCP 连接，所以称为半开。不过，如果目标端口是关闭的，半开扫描和全开扫描倒是没有区别，因为会收到一个 RST 响应，告知该端口未开放。

在半开扫描中，考虑一种更为复杂的情况 那就是目标端口的 filtered 状态。这种状态往往是由包过滤机制造成的，过滤可能来自专业的防火墙设备、路由器规则或者主机上的软件防火墙 这种情况下会让扫描工作变得很难，因为这些端口几乎不提供任何信息。不过有时候它们也会响应 ICMP 错误消息，但更多时候包过滤机制不做出任何响应。这样就将没有收到回应数据包的端口归于 filtered 状态。另外，考虑收到 ICMP 错误消息的情形，这种情况下，也要将目标端口归于 filtered 状态。

当 TCP 连接的数据包被屏蔽时，一般会返回如下表所示的几种 ICMP 错误消息：

|类型(TYPE)|代码(CODE)| 意义|
|:--:|:--:|:--:|
|3| 1|主机不可达|
|3| 2 |协议不可达|
|3| 3 |端口不可达|
|3| 9 |目的网络被强制禁止|
|3| 10 |目的主机被强制禁止|
|3| 13 |由于过滤，通信被强制禁止|


## 5.4 服务扫描

在对目标进行渗透测试的时候，要尽量地检测出目标系统运行的各种服务。对于入侵者来说，发现这些运行在目标上的服务，就可以利用这些软件上的漏洞入侵目标；对于网络安全的维护者来说，也可以提前发现系统的漏洞，从而预防这些入侵行为。

有一些扫描工具采用了抓取软件 banner 的方法，因为很多的软件都会在连接之后提供一个表明自身信息的 banner 可以编写程序来抓取这个 banner 从中读出目标软件的信息，这是一个比较不错的方法。

最为优秀的一种方法，就是向目标开放的端口 发送探针数据包，然后根据返回的数据包与数据库中的记录进行比对，找出具体的服务信息。著名的 Nmap 扫描工具就是
采用了这种方法，它包含一个十分强大的 Nmap-service-prob 数据库，这个库中包含世界上大部分常见软件的信息，而且这个库还在完善中。

## 5.5 操作系统扫描

几乎所有的工具都使用了一种“猜”的方法,当然这不是凭空的猜测，目前远程对操作系统进行检测的方法一般可以分成以下两类

1)被动式方法：这种方法是通过抓包工具来收集流经网络的数据包，再从这些数据包中分析出目标主机的操作系统信息

2)主动式方法：向目标主机发送特定的数据包，目标主机一般会对这些数据包做出回应，对这些回应做出分析，就有可能得知远程主机的操作系统类型。这些信息可以是正常的
网络程序如：Telnet、FTP 等与主机交互的数据包，也可以是一些经过精心构造、正常的或残缺的数据包。